<a href="https://colab.research.google.com/github/KevinrealG/Series_de_tiempo/blob/main/02_Aplying_arima_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Datos y Librerias

In [1]:
import pandas as pd
import numpy as np

Fremont Bridge Sidewalks, south of N 34th St
!curl -o FremontBridge.csv https://data.seattle.gov/api/views/65db-xm6k/rows.csv?accessType=DOWNLOAD

The Fremont Bridge Bicycle Counter began operation in October 2012 and records the number of bikes that cross the bridge using the pedestrian/bicycle pathways. Inductive loops on the east and west pathways count the passing of bicycles regardless of travel direction. The data consists of a date/time field: Date, east pathway count field: Fremont Bridge NB, and west pathway count field: Fremont Bridge SB. The count fields represent the total bicycles detected during the specified one hour period. Direction of travel is not specified, but in general most traffic in the Fremont Bridge NB field is travelling northbound and most traffic in the Fremont Bridge SB field is travelling southbound.


In [ ]:
data =

# Arima

Flow Diagram for ARIMA Time Series Analysis:
1. Data Preparation:
Gather the time series data and ensure it's in a suitable format.
Check for missing values and handle them if present.
2. Check for Seasonality:
Visualize the time series to observe any repetitive patterns or cycles. Use a decomposition plot to identify trend, seasonality, and residual components.
3. Assess Stationarity:
Plot the time series to visually inspect trends, rolling statistics, and stationarity.
Perform the Augmented Dickey-Fuller (ADF) test to check for stationarity.
4. Differencing:
Apply differencing to achieve stationarity if the series is non-stationary.
Check stationarity after differencing using the ADF test.
5. ACF and PACF Analysis:
Examine the Autocorrelation Function (ACF) and Partial Autocorrelation Function (PACF) of the differenced series to determine the orders (p, d, q) for ARIMA.
6. Fit ARIMA Model:
Fit the ARIMA model to the differenced series using the determined orders (p, d, q).
Generate forecasts in the differenced space.
7. Inverse Differencing for Interpretation:
Invert the differencing process to translate the forecasts back to the original scale.
Apply cumulative sum to the forecasted differences to obtain predictions in the original scale.
8. Evaluate and Refine:
Assess the model's performance using appropriate evaluation metrics (e.g., MAE, RMSE).
Refine the model if necessary by adjusting parameters and re-fitting.

# LSTM